# Combining models: Base on different data columns to combine different models and build an mix model

In [22]:
# 1 = logistic Regression
# 2 = SVM
# 3 = Random Forest
# 4 = Knn
# 5 = MLP
# 6 = LinearSVM
model1 = 1
model2 = 2

# Use the variable here to modify the combination

## Read Data and change rent to labels + choose the fields we need

In [23]:
import numpy as np
import pandas as pd
import math


data = pd.read_csv("dataMA6.csv")


data["RENT"] = np.where(data["RENT"].between(0,1300), 0, data["RENT"])
data["RENT"] = np.where(data["RENT"].between(1300,1900), 1, data["RENT"])
data["RENT"] = np.where(data["RENT"].between(1900,3000), 2, data["RENT"])
data["RENT"] = np.where(data["RENT"]>3000, 4, data["RENT"])
y=data["RENT"]



X = data[['CITY','SEX','AGE','EMPSTATD','SCHOOL','GRADEATT','FAMSIZE','EDUC','EMPSTAT','INCTOT','MORTGAGE','KITCHEN','FRIDGE','FTOTINC','MARST','RACE', 'PHONE','CINETHH']]
#print(X.shape)
X['CITY'] = X['CITY'].replace(870, 1)
X['CITY'] = X['CITY'].replace(810, 0)
X['CITY'] = X['CITY'].replace(930, 2)
#print(X.head(5))
X = pd.DataFrame(X,dtype=np.int)
X = X.dropna(axis=0)



F:\anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

## Scale the dataset and split train and test data

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#X1 = X[['CITY','SEX','SCHOOL','EDUC','GRADEATT','EMPSTAT']]
#X1 = X[["CITY","INCTOT","AGE"]]

from sklearn.model_selection import train_test_split
X_train1,X_test1, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train1[["INCTOT",'AGE']]
X_test = X_test1[["INCTOT",'AGE']]


scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)



## Start Layer1 models for [["INCTOT","AGE"]]

### RamdomForest

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
#print(rf.score(X_train, y_train))
ret = rf.predict(X_train)
ret1 = rf.predict(X_test)
#print(ret[:5],y[:5])
score = sum((ret-y_train)**2)
#print(score)

### MLP

In [26]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15,7), random_state=1, max_iter=3000) 
clf.fit(X_train, y_train)
#print(clf.score(X_train, y_train))
pred = clf.predict(X_train)
pred0 = clf.predict(X_test)
#print(pred[:10],y_train[:10])
score = sum((pred-y_train)**2)
#print(score)

### Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(max_iter=2000).fit(X_train, y_train)
#print(reg.score(X_train, y_train))
pred2 = reg.predict(X_train)
#print(pred2[:5])
pred5 = reg.predict(X_test)
score = (pred5==y_test).astype(int).mean()
#print(score)

### SVM

In [28]:
from sklearn.svm import SVC

clf = SVC(gamma='auto',C=1)
clf.fit(X_train, y_train)
#print(clf.score(X_train, y_train))
pred3 = clf.predict(X_train)
#print(pred3[:5])
#print(y[:5])
score = (pred3==y_train).astype(int).mean()
pred4 = clf.predict(X_test)
score = (pred4==y_test).astype(int).mean()
#print(score)

### Linear SVM

In [29]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=1, max_iter = 20000)
clf.fit(X_train, y_train)
#print(clf.score(X_train, y_train))
pred8 = clf.predict(X_train)
#print(pred3[:5])
#print(y[:5])
score = (pred8==y_train).astype(int).mean()
pred9 = clf.predict(X_test)
score = (pred9==y_test).astype(int).mean()
#print(score)

### KNN

In [30]:
from sklearn.neighbors import KNeighborsClassifier

def knn(X_train,X_test,y_train,y_test,k):
    """knn to train and return accuracy"""
    #scaler = StandardScaler()
    #X_train = scaler.fit_transform(X_train)
    #X_test = scaler.fit_transform(X_test)
    model = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    model.fit(X_train,y_train)
    pred = model.predict(X_train)
    train_acc = (pred==y_train).astype(int).mean()
    pred = model.predict(X_test)
    test_acc = (pred==y_test).astype(int).mean()
    print("k="+ str(k))
    print("train accuracy="+ str(train_acc))
    print("test accuracy="+ str(test_acc))
    return train_acc,test_acc

#for i in range(100,150):
#    knn(X_train,X_test,y_train,y_test,i)
    
    
model = KNeighborsClassifier(n_neighbors=62, n_jobs=-1)
model.fit(X_train,y_train)
pred6 = model.predict(X_train)
train_acc = (pred6==y_train).astype(int).mean()
pred7 = model.predict(X_test)
test_acc = (pred7==y_test).astype(int).mean()
#print(test_acc)

## Start Model layer2 for 
[['CITY','SEX','SCHOOL','EDUC','GRADEATT','EMPSTAT','FAMSIZE','MORTGAGE','KITCHEN','FRIDGE','MARST','RACE', 'PHONE','CINETHH']]

### Randomforest

In [31]:
#X2 = X[["CITY","INCTOT","AGE"]]
#X2 = X[["INCTOT"]]

X2 = X_train1[['CITY','SEX','SCHOOL','EDUC','GRADEATT','EMPSTAT','FAMSIZE','MORTGAGE','KITCHEN','FRIDGE','MARST','RACE', 'PHONE','CINETHH']]
X3 = X_test1[['CITY','SEX','SCHOOL','EDUC','GRADEATT','EMPSTAT','FAMSIZE','MORTGAGE','KITCHEN','FRIDGE','MARST','RACE', 'PHONE','CINETHH']]
if model1 == 1:
    X2["PRED"] = pred2
    X3["PRED"] = pred5
elif model1 ==2:
    X2["PRED"] = pred3
    X3["PRED"] = pred4
elif model1 ==3:
    X2["PRED"] = ret
    X3["PRED"] = ret1
elif model1 ==4:
    X2["PRED"] = pred6
    X3["PRED"] = pred7
elif model1 ==5:
    X2["PRED"] = pred
    X3["PRED"] = pred0
elif model1 ==6:
    X2["PRED"] = pred8
    X3["PRED"] = pred9
else:
    X2["PRED"] = pred2
    X3["PRED"] = pred5
    
X2= scaler.fit_transform(X2)

X3= scaler.transform(X3)

def Model2_RF():
    reg = RandomForestClassifier(n_estimators=500, random_state=1, n_jobs=-1).fit(X2, y_train)
    #print(reg.score(X2, y_train))
    pred = reg.predict(X3)

    #print(pred[:5])
    #print(y_test[:5])
    score = (pred==y_test).astype(int).mean()
    print("accuracy=",score)



F:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
F:\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### SVM

In [32]:
def model2_svm():
    clf = SVC(gamma='auto',C=1.3)
    clf.fit(X2, y_train)
    predfinal = clf.predict(X2)
    score = (predfinal==y_train).astype(int).mean()
    #print(score)
    predfinal = clf.predict(X3)
    score = (predfinal==y_test).astype(int).mean()
    print("accuracy=",score)

### Logistic Regression

In [33]:
def model2_Logistic():
    clf = LogisticRegression(max_iter = 10000)
    clf.fit(X2, y_train)
    predfinal = clf.predict(X3)
    score = (predfinal==y_test).astype(int).mean()
    print("accuracy=",score)

### Linear SVM

In [34]:

from sklearn.svm import LinearSVC
def model2_linearsvm():
    clf = LinearSVC(C=3, max_iter = 10000)
    clf.fit(X2, y_train)
    predfinal = clf.predict(X2)
    score = (predfinal==y_train).astype(int).mean()
    #print(score)
    predfinal = clf.predict(X3)
    score = (predfinal==y_test).astype(int).mean()
    print("accuracy=",score)

In [35]:
def model2_knn():  
    model = KNeighborsClassifier(n_neighbors=62, n_jobs=-1)
    model.fit(X2,y_train)

    knnpredict = model.predict(X3)
    test_acc = (knnpredict==y_test).astype(int).mean()
    print("accuracy=",test_acc)

# Result:

In [36]:
if model1 == 1:
    print("Model1: Logistic Regression")
elif model1 ==2:
    print("Model1: SVM")
elif model1 ==3:
    print("Model1: Random Forest")
elif model1 ==4:
    print("Model1: KNN")
elif model1 ==5:
    print("Model1: MLP")
elif model1 ==6:
    print("Model1: LinearSVM")
else:
    print("Model1:Wrong Number")

if model2 == 1:
    print("Model2: Logistic Regression")
    model2_Logistic()
elif model2 ==2:
    print("Model2: SVM")
    model2_svm()
elif model2 ==3:
    print("Model2: Random Forest")
    Model2_RF()
elif model2 ==4:
    print("Model2: KNN")
    model2_knn()
elif model2 ==5:
    print("Model2: MLP")
    print("Not supported yet")
elif model2 ==6:
    print("Model2: LinearSVM")
    model2_linearsvm()
else:
    print("Model2:Wrong Number")

Model1: Logistic Regression
Model2: KNN
accuracy= 0.6129963898916968


The best result we get is: <br>
    SVM+LogisticRegression = 0.6194945848375452 <br>
    LogisticRegression+SVM = 0.6274368231046932 <br>